In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("results/model_results.csv")

Como o número de épocas se relaciona com o learning_rate

- complexidade do modelo 
- learning rate
- época

+complexidade -> mais épocas, learning rate menor (supondo que não ha minimos locais)
              -> menor épocas, learning rate maior

# **Explorando** 

In [ ]:
# a complexidade do modelo é definida pelo número de parâmetros
def model_complexity(x):
    lista = eval(x)
    lista.insert(0, 16)
    return sum(i*j for i, j in zip(lista, lista[1:]))

df = df.assign(complexity = df.hidden_layers.apply(model_complexity))
df.hidden_layers = df.hidden_layers.apply(eval)
df = df.assign(min_neuronio_layer = df.hidden_layers.apply(min),
               complexity_tier = pd.cut(df.complexity, 3).cat.codes.map({0:"baixo", 1:"médio", 2:"alto"}))


In [ ]:
plot_df = df.rename(columns={"accuracy":"Acurácia", 
                             "complexity_tier": "Rank de Complexidade",
                             "learning_rate": "Taxa de Aprendizagem",
                             "num_epochs":"Número de épocas",
                             "min_neuronio_layer":"Menor Camada"})

g = sns.FacetGrid(plot_df, row="Rank de Complexidade", hue="Número de épocas", height=4, aspect=2,palette="bright")
g.map(sns.stripplot, 
      "Taxa de Aprendizagem", 
      "Acurácia", 
#      alpha=0.4,
      edgecolor="black",
      linewidth=0.8, 
      jitter=0.15,
      order=[0.001, 0.01, 0.1, 0.15])

g.map(plt.axhline, y=0.8, ls='--', c='gray')
g.map(plt.axhline, y=0.4, ls='--', c='gray')
g.map(plt.axhline, y=0.2, ls='--', c='gray')

g.map(plt.axhspan, ymin=.8, ymax=1, color='lightgray')
g.map(plt.axhspan, ymin=.4, ymax=.8, color='beige')
g.map(plt.axhspan, ymin=.2, ymax=.4, color='peachpuff')

g.add_legend()
sns.move_legend(g, "lower center", bbox_to_anchor=(.45, 1.01), ncol=4, title="Número de Épocas", frameon=False)

for lh in g._legend.legendHandles: 
    lh._sizes = [100]
    
plt.savefig("figures/analysis.eps", format="eps", bbox_inches='tight')

# **Análise Imagem**

In [ ]:
metrics = [np.mean, np.std, min, max]
plot_df.groupby("Menor Camada").agg({"Acurácia":metrics})

In [ ]:
regiao3 = plot_df.query("0.2 < Acurácia & Acurácia < 0.4")
camadas3 = regiao3["Menor Camada"].value_counts().sort_index()
camadas3.name = "Região 3"

In [ ]:
regiao2 = plot_df.query("0.4 < Acurácia & Acurácia < 0.8")
camadas2 = regiao2["Menor Camada"].value_counts().sort_index()
camadas2.name = "Região 2"

In [ ]:
regiao1 = plot_df.query("Acurácia > 0.8")
regiao1["Menor Camada"].value_counts().sort_index()
camadas1 = regiao1["Menor Camada"].value_counts().sort_index()
camadas1.name = "Região 1"